# Исследование надежности заемщиков

## 1 Откроем таблицу и изучим общую информацию о данных

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

### Выведем первые 30 строчек датафрейма на экран

In [2]:
data.head(30)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Выведем основную информацию о датафрейме

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## 2 Предобработка данных

### 2.1 Удаление пропусков

#### Выведем количество пропущенных значений для каждого столбца

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**`total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`.**

In [11]:
for itype in data['income_type'].unique():
    data.loc[(data['income_type'] == itype) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == itype), 'total_income'].median()

**Заполним пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`**

In [13]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Проверим, заполнили ли все пропуски**

In [14]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### 2.2 Обработка аномальных значений

**Заменим все отрицательные значения в столбце `days_employed` (трудовой стаж)**

In [6]:
data['days_employed'] = data['days_employed'].abs()

**Для каждого типа занятости выведем медианное значение трудового стажа в днях**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

**Обработаем столбец `children`**

Выведем уникальные значения в данном столбце

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

Видим 2 аномальных значения (-1 и 20), значит удаляем строки, в которых встречаются данные значения.

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Проверим столбец children

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### 2.3 Изменение типов данных

**Заменим вещественный тип данных в столбце `total_income` на целочисленный для удобства просмотра данных**

In [15]:
data['total_income'] = data['total_income'].astype(int)

### 2.4 Обработка дубликатов

**Просматривая столбец `education` видим, что есть одни и те же значения, записанные в разных регистрах. Обработаем эти значения**

Приведем данные в столбце к нижнему регистру

In [16]:
data['education'] = data['education'].str.lower()

Просмотрим количество строк-дубликатов в столбце и удалим их.

In [17]:
data.duplicated().sum()

71

In [18]:
data = data.drop_duplicates()

In [19]:
data.duplicated().sum()

0

### 2.5 Категоризация данных

**Создадим в датафрейме `data` столбец `total_income_category` с категориями дохода:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


In [20]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [21]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [22]:
data['total_income_category'].head()

0    B
1    C
2    C
3    B
4    C
Name: total_income_category, dtype: object

**Выведем перечень уникальных целей взятия кредита (столбец `purpose`)**

In [23]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [24]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [25]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

In [26]:
data['purpose_category'].head()

0    операции с недвижимостью
1      операции с автомобилем
2    операции с недвижимостью
3       получение образования
4          проведение свадьбы
Name: purpose_category, dtype: object

## 3. Исследуем данные и ответим на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

Найдем зависимость между кол-вом детей (`children`) и возвратом кредита в срок(`debt`)
Для этого сделаем новую таблицу `debt_children_correlation`


In [27]:
debt_children_correlation = data[['children', 'debt']].copy()
debt_children_correlation

,children,debt
0,1,0
1,1,0
2,0,0
3,3,0
4,0,0
...,...,...
21520,1,0
21521,0,0
21522,1,1
21523,3,1


Рассмотрим все уникальные значения в столбце `children`

In [28]:
debt_children_correlation['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

Подразделим семьи на 3 категории:
<ul>
    <li>Без детей (0)</li>
    <li>Малодетная семья (1-2)</li>
    <li>Многодетная семья (>3)</li>
</ul>

Для этого создаем функцию `categorize_family()`
   

In [29]:
def categorize_family(row):
    try:
        if (row == 0):
            return 'Без детей'
        elif (1 <= row <= 2):
            return 'Малодетная семья'
        elif row >= 3:
            return 'Многодетная семья'
    except:
        return 'нет категории'

In [30]:
debt_children_correlation['family_category'] = debt_children_correlation['children'].apply(categorize_family)

In [31]:
debt_children_correlation

,children,debt,family_category
0,1,0,Малодетная семья
1,1,0,Малодетная семья
2,0,0,Без детей
3,3,0,Многодетная семья
4,0,0,Без детей
...,...,...,...
21520,1,0,Малодетная семья
21521,0,0,Без детей
21522,1,1,Малодетная семья
21523,3,1,Многодетная семья


Сгруппируем таблицу по столбцу `family_category` и вычислим, какая категория семей чаще всего имеет задолженность по возврату кредита

In [32]:
debt_grouped = debt_children_correlation.groupby('family_category').agg({'debt': ['count', 'sum']})
debt_grouped

debt      
                   count   sum
family_category               
Без детей          14091  1063
Малодетная семья    6860   638
Многодетная семья    380    31

Посчитаем среднее количество задолженностей по каждой категории

In [33]:
debt_grouped['mean'] = debt_grouped[('debt',   'sum')] / debt_grouped[('debt', 'count')] * 100
debt_grouped.sort_values(by='mean', ascending=False)

debt            mean
                   count   sum          
family_category                         
Малодетная семья    6860   638  9.300292
Многодетная семья    380    31  8.157895
Без детей          14091  1063  7.543822

#### Вывод

Клиенты без детей, в целом, чаще возвращают кредиты в срок, скорее всего это из-за меньших трат. Как мы можем увидеть, средние значения почти не отличаются. Соответственно можно сделать вывод, что корреляции между количеством детей и возвратом кредита в срок нет.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

Рассмотрим все уникальные значения в столбце `family_status`

In [34]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Создадим таблицу с столбцаи `family_status`, `debt`

In [35]:
family_status_debt = data[['family_status', 'debt']]
family_status_debt

,family_status,debt
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,гражданский брак,0
...,...,...
21520,гражданский брак,0
21521,женат / замужем,0
21522,гражданский брак,1
21523,женат / замужем,1


Сгруппируем данные по значениям из столбца `family_status` и посчитаем количество должников в каждой категории

In [36]:
family_status_debt_grouped = family_status_debt.groupby('family_status').agg({'debt': ['count', 'sum']})
family_status_debt_grouped

debt     
                       count  sum
family_status                    
Не женат / не замужем   2796  273
в разводе               1189   84
вдовец / вдова           951   63
гражданский брак        4134  385
женат / замужем        12261  927

Найдем средний процент должников в каждой категории

In [40]:
family_status_debt_grouped['mean'] = (family_status_debt_grouped[('debt', 'sum')] /
                                      family_status_debt_grouped[('debt', 'count')] * 100)

In [41]:
family_status_debt_grouped.sort_values(by='mean', ascending=False)

debt           mean
                       count  sum          
family_status                              
Не женат / не замужем   2796  273  9.763948
гражданский брак        4134  385  9.313014
женат / замужем        12261  927  7.560558
в разводе               1189   84  7.064760
вдовец / вдова           951   63  6.624606

#### Вывод

На основе полученных данных можно выделить, что неженатые и состоящие в гражданском браке чаще являются должниками. Предполагаю, что на это есть несколько причин: низкий доход, молодой возраст, низкая ответственность. Можно сказать, что есть прямая зависимость семейного положения и возвратом кредита в срок.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Рассмотрим уникальные значения в столбце `total_income_category`

In [42]:
data['total_income_category'].unique()

array(['B', 'C', 'D', 'E', 'A'], dtype=object)

Создадим новую таблицу с столбцами `total_income_category`, `debt`

In [43]:
total_income_debt = data[['total_income_category', 'debt']].copy()
total_income_debt

,total_income_category,debt
0,B,0
1,C,0
2,C,0
3,B,0
4,C,0
...,...,...
21520,B,0
21521,C,0
21522,C,1
21523,B,1


Сгруппируем данные по значениям из столбца `total_income_category` и посчитаем количество должников в каждой категории

In [44]:
total_income_debt_grouped = total_income_debt.groupby('total_income_category').agg({'debt':['count', 'sum']})
total_income_debt_grouped

debt      
                       count   sum
total_income_category             
A                         25     2
B                       5014   354
C                      15921  1353
D                        349    21
E                         22     2

Посчитаем средний процент должников в каждой категории

In [47]:
total_income_debt_grouped['mean'] = (total_income_debt_grouped[('debt', 'sum')] / 
                                     total_income_debt_grouped[('debt','count')] * 100)


In [48]:
total_income_debt_grouped.sort_values(by='mean', ascending=False)

debt            mean
                       count   sum          
total_income_category                       
E                         22     2  9.090909
C                      15921  1353  8.498210
A                         25     2  8.000000
B                       5014   354  7.060231
D                        349    21  6.017192

#### Вывод

Рассмотрим категории `C` (доход от 50к до 200к) и `B` (доход от 200к до 1кк). Остальные категории рассматривать не целесообразно, тк мало данных и высока вероятность погрешности. В категорию `B` можно отнести клиентов, которые занимаюстя предпринимательством. Соответственно, можно предположить, что это состоявшиеся люди, у которых развито чувство ответственности, и поэтому они чаще возвращают кредит в срок. У категории `C` меньший доход, и большая финансовая нагрузка, поэтому они чаще становятся должниками.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

Рассмотрим уникальные значения в столбце `purpose_category`

In [50]:
data['purpose_category'].unique()

array(['операции с недвижимостью', 'операции с автомобилем',
       'получение образования', 'проведение свадьбы'], dtype=object)

Создадим новую таблицу с столбцами `purpose_category`, `debt`

In [51]:
purpose_debt = data[['purpose_category','debt']].copy()
purpose_debt

,purpose_category,debt
0,операции с недвижимостью,0
1,операции с автомобилем,0
2,операции с недвижимостью,0
3,получение образования,0
4,проведение свадьбы,0
...,...,...
21520,операции с недвижимостью,0
21521,операции с автомобилем,0
21522,операции с недвижимостью,1
21523,операции с автомобилем,1


Сгруппируем данные по значениям из столбца `purpose_category` и посчитаем количество должников в каждой категории

In [52]:
purpose_debt_grouped = purpose_debt.groupby('purpose_category').agg({'debt':['sum', 'count']})
purpose_debt_grouped

debt       
                          sum  count
purpose_category                    
операции с автомобилем    400   4279
операции с недвижимостью  780  10751
получение образования     369   3988
проведение свадьбы        183   2313

Посчитаем средний процент должников в каждой категории

In [53]:
purpose_debt_grouped['mean'] = (purpose_debt_grouped[('debt','sum')] /
                                purpose_debt_grouped[('debt','count')] * 100)

In [54]:
purpose_debt_grouped.sort_values(by='mean', ascending=False)

debt             mean
                          sum  count          
purpose_category                              
операции с автомобилем    400   4279  9.347978
получение образования     369   3988  9.252758
проведение свадьбы        183   2313  7.911803
операции с недвижимостью  780  10751  7.255139

#### Вывод

Здесь прослеживается прямая корреляция между целью кредита и возвратом его в срок. Меньше всего должников в категории операции с недвижимостью. Обосновывается это тем, что люди, которые берут кредит на операции с недвижимостью более ответственны и мотивированы вернуть его в срок, такая же история с теми, кто берет кредит на проведение свадьбы.

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

Часто причиной может быть человеческий или технический фактор.Возможно, сами клиенты не предоставили информацию, либо для каких-то категорий не запрашивают эти данные. Касательно технической части: проблемы могли возникнуть на этапе загрузки/выгрузки данных. Либо при формировании запроса к БД.

## 4. Общий вывод

Рассматривая каждую гипотезу не могу сказать, что есть четко прослеживаемая взаимосвязь, т.к. корреляция относительно невысокая. Но, мы смогли получить примерную картину человека, который скорее всего не вернет кредит в срок: низкий доход, гражданский брак или вовсе не женат, а цель кредита: покупка чего-либо недорогостоящего. Высокую же вероятность возврата кредита будет иметь клиент, вдовец/вдова, с высоким доходом и без детей, с целью кредита - на недвижимость.

В проекте мы выполнили: Предобработку данных - обработали дубликаты и аномальные значения, устранили пропуски. Сделали функции для категоризирования целей кредита, уровня дохода, количества детей в семьях. Провели анализ влияния различных факторов на возвращения кредита в срок.

Касательно рекомендаций: Предоставить данные на большее кол-во клиентов, т.к. 3 из 5 категорий по доходам нецелесообразно анализировать из-за недостатка данных. Узнать причину пропусков в данных в столбцах total_income и days_employed у оператора БД.